* Ссылки 
  * [Гистограммы](https://habr.com/ru/post/470535/).<br>
  * [Использование pandas+sklearn](https://habr.com/ru/post/202090/).<br>
  * [Kaggle и Titanic](https://github.com/sumitmukhija/Titanic/blob/master/Tip%20of%20the%20Iceberg.ipynb).<br>
  * [Титаник на Kaggle для начинающих](https://neurohive.io/ru/osnovy-data-science/razbor-resheniya-zadachi-titanik-na-kaggle-dlya-nachinajushhih/).<br>
  * [Titanic Tutorial](https://www.kaggle.com/alexisbcook/titanic-tutorial).<br>

In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import plot,show,hist
from scipy.stats.kde import gaussian_kde
from numpy import linspace,hstack
from pylab import plot,show,hist
#%config InlineBackend.figure_format = 'svg' для большей четкости графиков
matplotlib.style.use('ggplot')
%matplotlib inline


os.chdir(r'C:\Users\Mr Alex\Documents\GitHub\Kaggle\Titanic')


IS_FOR_KAGGLE = False

if IS_FOR_KAGGLE:
    trd = pd.read_csv('../input/train.csv')
    tsd = pd.read_csv('../input/test.csv')
else:
    trd = pd.read_csv('train.csv')
    tsd = pd.read_csv('test.csv')

df = pd.concat([trd, tsd], ignore_index=True, sort  = False)

#Сливаем открытые фреймы в один
#td = pd.concat([df, tsd], ignore_index=True, sort = False)

In [ ]:
#Общие данные 
df.info()
print('Текущая длина фрейма: ', len(df))

#Смотрим коэффициенты корелляций. Мало больших значений - плохо для факторного анализа
df.corr()

In [ ]:
#Смотрим коэффициенты корелляций. Мало больших значений - плохо для факторного анализа
df.corr()

In [ ]:
#Преобразуем объектные столбцы в численные. Колонку y тоже
df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
df['Sex'] = df['Sex'].map({'female': 0, 'male': 1})

In [ ]:
df['Relatives'] = df['SibSp'] + df['Parch']

In [ ]:
#Стандартизация данных
from sklearn import preprocessing

#Построение диаграмм рассеивания
from pandas.plotting import scatter_matrix

#Графика для интерпретации моделей
from IPython.display import Image
from sklearn.tree import export_graphviz
from subprocess import call

#Деревья решений для задачи классификации
from sklearn.tree import DecisionTreeClassifier

#Деревья решений для задач регрессии 
from sklearn.ensemble import RandomForestClassifier

#Калибровка деревьев решений
from sklearn.calibration import CalibratedClassifierCV

In [ ]:
columns = ['Embarked']
df.drop(columns, inplace=True, axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

#Расщепление на обучающую и тестовую выборки

X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,
                                                    # доля объёма тестового множества
                                                    test_size=0.2)

In [ ]:
model = DecisionTreeClassifier(random_state=42,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='gini',
                               # максимальная глубина дерева
                               max_depth=5,
                               # минимальное число элементов в узле для разбиения (может быть долей)
                               min_samples_split=5,
                               # минимальное число элементов в листе (может быть долей)
                               min_samples_leaf=5,
                               # минимальное значение дельты impurity
                               # min_impurity_decrease=0,
                               # веса для классов (можно дополнительно штрафовать за ошибку в нужных классах).
                               # поддерживает опцию 'balanced'.
                               class_weight=None
                               
                              )

model.fit(X, y)

In [ ]:
export_graphviz(model,
                out_file='tree.dot',
                #задать названия фич
                #feature_names=X.columns,
                class_names=None,
                #показывать названия полей у численных значений внутри узла
                label='all',
                #раскрашивать узлы в цвет преобладающего класса
                filled=True,
                #показывать значение impurity для каждого узла
                impurity=True,
                #показывать номера узлов
                node_ids=True,
                #Показывать доли каждого класса в узлах (а не количество)
                proportion=True,
                #Повернуть дерево на 90 градусов (вертикальная ориентация)
                rotate=True,
                #Число точек после запятой для отображаемых дробей
                #precision=3
               )


In [ ]:
#Строим предсказание (обучающая, тестовая)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
y_pred_train = model.predict(X_train)
print(classification_report(y_train, y_pred_train))

#Смотрим точность(ошибки 1 и 2го рода)
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

#Cмотрим важность признаков
fi = pd.DataFrame({'features': X_train.columns, 'importance': model.feature_importances_})
fi.sort_values('importance', ascending=False).head(10)

In [ ]:
#Графика для интерпретации моделей
from IPython.display import Image
from sklearn.tree import export_graphviz
from subprocess import call

import pydot
#Преобразуем файл .dot в .png
#node - номер узла, X[1]<=1.5 правило расщепления, gini, samples-доля наблюдений попавших в узел, p-value (p0, pX)
(graph,) = pydot.graph_from_dot_file('tree.dot')
graph.write_png('tree.png')
Image("tree.png")

#Модель позволяет оценить ценность (importance) и эффективность каждой фичи, считая для каждой из сумму дельты H 
pd.DataFrame({'feature': X.columns,
              'importance': model.feature_importances_}).sort_values('importance', 
            ascending=False
            )


In [ ]:
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

#Строим предсказание модели на тестовом множестве
y_pred = model.predict(X_test)

#Оценка качества классификатора: доля совпавших ответов в y_pred и y_test, или считаем точность и полноту
#Если доля в обучающем выше тестового, означает переобученность модели. Нужно упрощать модель
#Матрица ошибок  𝐶=(𝑐𝑖,𝑗) , где  𝑐𝑖,𝑗 количество элементов класса 𝑖 , которым классификатор присвоил класс 𝑗 
#Точность(precision) - доля правильно классифицированных объектов в найденных классификатором. 
#Полнота(recall) - доля этих объектов НА САМОМ ДЕЛЕ
conf_mat = metrics.confusion_matrix(y_test, y_pred)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

In [ ]:
#Гармоническое среднее F1 = 2*точность*полнота/(точность+полнота). Считается с помощью classification_report
print(metrics.classification_report(y_pred, y_test))

In [ ]:
model = RandomForestClassifier(random_state=42, #зерно датчика случайных чисел
                               #число деревьев в лесу
                               n_estimators=30,
                               #функция для дельта H, impurity ('gini' или 'entropy')
                               criterion='gini',
                               #Макс число слоев
                               max_depth=5,
                               #Вычислять out-of-bag ошибку
                               oob_score=True,
                               #использовать результаты предыдущего вызова и нарастить предыдущий лес 
                               warm_start=False,
                               #веса классов для балансировки выборки для обучения
                               class_weight=None
                               
                              )

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(metrics.classification_report(y_pred, y_test))

print('Out-of-bag score: {0}'.format(model.oob_score_)) 

pd.DataFrame({'feature': X.columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)

In [ ]:
#Создаем и обучаем модель
model = GradientBoostingClassifier(random_state=42,
                                   # Число деревьев
                                   n_estimators=500,
                                   #загрязнение измеряем “mse”, “mae” или “friedman_mse” (mse с улучшениями)  
                                   criterion='friedman_mse', 
                                   #Максимальная глубина каждого дерева
                                   #критерий качества ‘deviance’ (кросс-энтропия) или ‘exponential’
                                   #‘deviance’ для классификации с вероятностями на выходе
                                   loss='deviance', 
                                   # минимальное уменьшение загрязнения 
                                   min_impurity_decrease=0.0, 
                                   # Устарело
                                   min_impurity_split=None,
                                   # число узлов в дереве
                                   max_depth=5,
                                   #минимальное число наблюдений в потомке
                                   min_samples_leaf=5, 
                                   #минимальное число наблюдений в родителе
                                   min_samples_split=10,
                                   #Параметр, уменьшающий переобучение, являющемся весом отдельного дерева (меньше лучше)
                                   learning_rate=0.01                                   
                                   )

model.fit(X_train, y_train)

#Строим предсказание (обучающая, тестовая)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
y_pred_train = model.predict(X_train)
print(classification_report(y_train, y_pred_train))

#Смотрим точность(ошибки 1 и 2го рода)
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

#Cмотрим важность признаков
fi = pd.DataFrame({'features': X_train.columns, 'importance': model.feature_importances_})
fi.sort_values('importance', ascending=False).head(10)

#Калибровка (интерпретация вероятности)
from sklearn.calibration import CalibratedClassifierCV
model_sigmoid = CalibratedClassifierCV(model, cv=2, method='sigmoid')
# method : ‘sigmoid’ or ‘isotonic’

# Calibrate probabilities
model_sigmoid.fit(X_train, y_train)

# View calibrated probabilities
model_sigmoid.predict_proba(X_test)[0:11, :]

In [ ]:
#Статистика по всем
df.describe(include="all")

In [ ]:
#Только по цифровым колонкам ищем нужное нам значение
df.mean(numeric_only = True)

In [ ]:
#Тепловая карта пропущенных значений
df.isnull().sum()
sns.heatmap(td.isnull(), cbar = False).set_title("Missing values heatmap")

In [ ]:
#Преобразуем колонку с объектами в числовую
level_map = {"male": 1, "female": 0}
df['Sex'] = td['Sex'].map(level_map)

#Преобразуем Dtype колонки в bool
df['Survived'] = td['Survived'].astype('bool')

In [ ]:
#Статистика по родственникам на борту
df[['Parch', 'SibSp']].value_counts()

In [ ]:
#Оцениваем возраст
df["Age"].unique()
df["Age"].mean()
df_age = df['Age'].value_counts()
age_24_30 = df_age.loc[[24.00, 30.00]]
df.loc[:10, ["Age", "Sex"]]

In [ ]:
#Зависимая и независимые переменные. Сортировка выживших по полу и возрасту. 
df[["Survived", "Age", "Sex"]].value_counts().loc[1]

In [ ]:
#Зависимая и независимые переменные. Средняя доля выживших и среднее по возрасту и цене билета
df[["Survived", "Age", "Fare"]].mean(axis=0)

In [ ]:
#Получение сокращенного фрейма по конкретным критериям
df[df["Sex"].isin([True])].head()

In [ ]:
#Сортировка
df.sort_values('Age', ascending=False).head()

In [ ]:
#Агрегирование. Подсчет количества записей
df.groupby('Fare').count()

In [ ]:
#новые категории тарифов в новый столбец Fare_Category.
df['Fare_Category'] = pd.cut(td['Fare'], bins=[0,7.90,14.45,31.28,120], labels=['Low','Mid', 'High_Mid','High'])

In [ ]:
df.groupby('Survived')['Age'].plot.hist(density=True)
plt.legend()

In [ ]:
for column in ['Survived', 'Pclass', 'Sex']:
    print(df[column].value_counts(normalize=True))

In [ ]:
# Нужно, чтобы подписи к картинкам отображались корректно по-русски
matplotlib.rc('font', family='Arial')

In [ ]:
# Строим картинку
df['Survived'].value_counts().plot(kind='bar')
# Добавляем подпись
plt.title(u'Число выживших на Титанике')
plt.xticks([0, 1], ['Not Survived', 'Survived'], rotation=None)
plt.text(-0.1, 275, '61.5%', fontsize=15, color='silver')
plt.text(0.9, 150, '38.5%', fontsize=15, color='silver');

In [ ]:
#Доля выживших на Титанике
fig, ax = plt.subplots(1)
df['Survived'].value_counts().plot(kind='pie', labels=['', ''],
                                   ax=ax, autopct='%.2f%%', fontsize=15)
ax.set_title(u'Доля выживших на Титанике')
ax.axis('equal')
ax.legend(labels=[u'Выжили', u'Погибли'], framealpha=0)
ax.set_ylabel('')
plt.tight_layout()

In [ ]:
df['Age'].plot(kind='box')
plt.title(u'Распределение возраста')

In [ ]:
ax = df.boxplot(column='Survived', by='Pclass')
# Хак для того, чтобы исправить наезжающие заголовки графика
ax.get_figure().suptitle('')

In [ ]:
#иерархический кластерный анализ
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster